sempre que adicionar ou mudar uma dependencia, usar pip freeze > requirements.txt


In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
import requests
import re

url = 'https://books.toscrape.com/catalogue/page-1.html'

response = requests.get(url) # resposta com o conteudo da pagina

result = response.text

soup = BeautifulSoup(result, 'lxml') # transofmr ao html da pagina em um objeto BeautifulSoup navegavel
print(soup.prettify()) # imprime o html formatado


In [ ]:
print(response)

In [ ]:
titles = [] # cria uma lista vazia para armazenar os titulos dos livros

tag_a = soup.find_all("a", attrs={"title":True}) # encontra todas as tags <a> com o atributo title
for a in tag_a: # itera sobre todas as tags <a> encontradas
    title = a.get('title') # extrai o valor do atributo title
    titles.append(title) # adiciona o valor a lista titles
    
print(titles) # imprime o valor do atributo title

#print(titles) # imprime a lista titles

#print(type(tag_a))
#print(tag_a)

In [ ]:
# função para transformar o nome das classes em numeros
def star_to_number(star):
    if star == 'One':
        return 1
    elif star == 'Two':
        return 2
    elif star == 'Three':
        return 3
    elif star == 'Four':
        return 4
    elif star == 'Five':
        return 5
    else:
        return 0

stars = [] # cria uma lista vazia para armazenar as estrelas

tag_p = soup.find_all("p", class_="star-rating") # encontra todas as tags <p> com a classe que contenha star-rating no nome da classe
for p in tag_p: # itera sobre todas as tags <p> encontradas
    star = p.get('class')[1] # extrai o valor do atributo class
    stars.append(star) # adiciona o valor a lista stars

for s in stars:
    s = star_to_number(s) # transforma o nome da classe em numero
    print(s) # imprime o valor


In [ ]:
prices = [] # lista para armazenar os preços

tag_p = soup.find_all("p", class_="price_color") # encontra todas as tags <p> com a classe que contenha star-rating no nome da classe
for p in tag_p: # itera sobre todas as tags <p> encontradas
    price = p.getText()
    price = price.replace('Â£', '$') # substitui o símbolo da libra por dólar
    prices.append(price) # adiciona o valor a lista prices
    print(price) # imprime o valor

print(prices) # imprime a lista prices


In [ ]:
dispo = [] # lista para armazenar a disponibilidade

tag_p = soup.find_all("p", class_="instock availability") # encontra todas as tags <p> com a classe instock availability
for p in tag_p: # itera sobre todas as tags <p> encontradas
    p = p.getText() # extrai o texto da tag <p>
    p = re.sub(r'\n+', ' ', p).strip() # remove os espaços em branco extras
    dispo.append(p) # adiciona o valor a lista dispo


print(dispo) # imprime a lista dispo

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
import requests
import re

# função para transformar o nome das classes em numeros
def star_to_number(star):
    if star == 'One':
        return 1
    elif star == 'Two':
        return 2
    elif star == 'Three':
        return 3
    elif star == 'Four':
        return 4
    elif star == 'Five':
        return 5
    else:
        return 0

dadosAcumulados = [] # lista para armazenar os dados acumulados

while response.status_code == 200:
    
    page = 1
    
    url = f'https://books.toscrape.com/catalogue/page-{page}.html'
    
    response = requests.get(url) # resposta com o conteudo da pagina
    
    result = response.text
    
    soup = BeautifulSoup(result, 'lxml')

    
    tag_a = soup.find_all("a", attrs={"title":True}) # encontra todas as tags <a> com o atributo title
    for a in tag_a: # itera sobre todas as tags <a> encontradas
        title = a.get('title') # extrai o valor do atributo title
        dadosAcumulados.append(title) # adiciona o valor a lista titles
    

    
    tag_p = soup.find_all("p", class_="star-rating") # encontra todas as tags <p> com a classe que contenha star-rating no nome da classe
    for p in tag_p: # itera sobre todas as tags <p> encontradas
        star = p.get('class')[1] # extrai o valor do atributo class, que é o segundo atributo
        dadosAcumulados.append(star) # adiciona o valor a lista stars
    

    
    tag_p = soup.find_all("p", class_="price_color") # encontra todas as tags <p> com a classe que contenha star-rating no nome da classe
    for p in tag_p: # itera sobre todas as tags <p> encontradas
        price = p.getText()
        price = price.replace('Â£', '$') # substitui o símbolo da libra por dólar
        dadosAcumulados.append(price) # adiciona o valor a lista prices
        #print(price) # imprime o valor


    
    tag_p = soup.find_all("p", class_="instock availability") # encontra todas as tags <p> com a classe instock availability
    for p in tag_p: # itera sobre todas as tags <p> encontradas
        dispo = p.getText() # extrai o texto da tag <p>
        dispo = re.sub(r'\n+', ' ', dispo).strip() # remove os espaços em branco extras
        dadosAcumulados.append(dispo) # adiciona o valor a lista dispo

    #contatenar todos os dados em um unico dataframe

    dadosAcumulados.append((title, star, price, dispo)) # adiciona os dados acumulados em uma lista
    
    
    page = page + 1



    

In [ ]:
dadosAcumulados = [] # lista para armazenar os dados acumulados

page = 1
    
url = f'https://books.toscrape.com/catalogue/page-{page}.html'
    
response = requests.get(url) # resposta com o conteudo da pagina

result = response.text

soup = BeautifulSoup(result, 'lxml')


tag_a = soup.find_all("a", attrs={"title":True}) # encontra todas as tags <a> com o atributo title
for a in tag_a: # itera sobre todas as tags <a> encontradas
    title = a.get('title') # extrai o valor do atributo title
    #dadosAcumulados.append(title) # adiciona o valor a lista titles



tag_p = soup.find_all("p", class_="star-rating") # encontra todas as tags <p> com a classe que contenha star-rating no nome da classe
for p in tag_p: # itera sobre todas as tags <p> encontradas
    star = p.get('class')[1] # extrai o valor do atributo class, que é o segundo atributo
    #dadosAcumulados.append(star) # adiciona o valor a lista stars



tag_p = soup.find_all("p", class_="price_color") # encontra todas as tags <p> com a classe que contenha star-rating no nome da classe
for p in tag_p: # itera sobre todas as tags <p> encontradas
    price = p.getText()
    price = price.replace('Â£', '$') # substitui o símbolo da libra por dólar
    #dadosAcumulados.append(price) # adiciona o valor a lista prices
    #print(price) # imprime o valor



tag_p = soup.find_all("p", class_="instock availability") # encontra todas as tags <p> com a classe instock availability
for p in tag_p: # itera sobre todas as tags <p> encontradas
    dispo = p.getText() # extrai o texto da tag <p>
    dispo = re.sub(r'\n+', ' ', dispo).strip() # remove os espaços em branco extras
    #dadosAcumulados.append(dispo) # adiciona o valor a lista dispo



dadosAcumulados.append((title, star, price, dispo)) # adiciona os dados acumulados em uma lista



In [34]:
for i in dadosAcumulados:
    print(i)

("It's Only the Himalayas", 'Two', '$45.17', 'In stock')


In [ ]:

# Inicializar a SparkSession
spark = SparkSession.builder \
    .appName("Exemplo Simples de Tabela") \
    .getOrCreate()

# Definir o schema da tabela
schema = StructType([
    StructField("id", IntegerType(), nullable=False),
    StructField("nome", StringType(), nullable=True),
    StructField("idade", IntegerType(), nullable=True),
    StructField("cidade", StringType(), nullable=True)
])

# Dados de exemplo
data = [
    (1, "João Silva", 28, "São Paulo"),
    (2, "Maria Souza", 34, "Rio de Janeiro"),
    (3, "Carlos Oliveira", 22, "Belo Horizonte"),
    (4, "Ana Santos", 45, "Curitiba")
]

# Criar DataFrame
df = spark.createDataFrame(data, schema=schema)

# Mostrar a tabela
print("Mostrando a tabela completa:")
df.show()

# Mostrar a tabela com formatação
print("\nMostrando a tabela com formatação:")
df.show(truncate=False)

# Mostrar o schema da tabela
print("\nSchema da tabela:")
df.printSchema()

spark.stop()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

# Criar SparkSession
spark = SparkSession.builder.appName("EmptyDF").getOrCreate()

# 1. Criar DataFrame vazio com schema definido
schema = StructType([])  # Schema vazio
df = spark.createDataFrame([], schema)

# 2. Adicionar colunas uma a uma
from pyspark.sql.functions import lit

# Adicionar coluna de strings
df = df.withColumn("nome", lit(None).cast(StringType()))

# Adicionar coluna de inteiros
df = df.withColumn("idade", lit(None).cast(IntegerType()))

# Adicionar coluna de doubles com valor padrão 0.0
df = df.withColumn("salario", lit(0.0).cast(DoubleType()))

df.printSchema()
df.show()